In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as colors
import numpy as np
import powerlaw
from tree import Tree
from fire import Fire
from forest import Forest
from analysis import Analyse
import pandas as pd
from sensitivity_analysis import SensitivityAnal
# from forest_sim import run_simulation, initialize_forest


A first step in our work is to show that our basic model based on Turcotte (1999) does indeed show Self-Organized Criticality. In other words: (1) it has a steady input, (2) there are 'avalanches' of output of size proportional tot hat of the system and (3) the system remains at a quasi-equilibrium state. 

(1) This is determined by the design of the model, per each time step there is between 0 and 1 new trees planted. 

(2) This is shown by comparing the distribution of fire sizes (i.e. the output of the system) to a power law distribution 

(3) This is deduced from the fact that, after a period of growth in number of trees, reaches a stable number around which it oscillates. 

Ilia Work 
____________

In [ ]:
L = 50
f = 50
timesteps = 10**5
instances = 20
freeze_time_during_fire = False
remember_history = False
analysis_1 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_1.run_all()


In [ ]:
# (2)
analysis_1.find_best_fitting_distributions()
print(f'{analysis_1.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

analysis_1.log_log_plot(scatter=False)

In [ ]:
# (3)
proportion_stable = analysis_1.find_proportion_stable(0.01)
print(f'{proportion_stable * 100}% of the model instances converge to a quasi equilibrium state')
analysis_1.plot_number_trees_timeseries()

## Comparing time-freezing with linear time

9

In [ ]:
L = 50
f = 19

# Scaled down from 10 million
timesteps = 1_000_000
instances = 1
freeze_time_during_fire = False
remember_history = False

analysis_3 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_3.run_all()

analysis_3.find_best_fitting_distributions()
print(f'{analysis_3.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


f = 76

analysis_4 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_4.run_all()

analysis_4.find_best_fitting_distributions()
print(f'{analysis_4.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

f = 305

analysis_5 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_5.run_all()

analysis_5.find_best_fitting_distributions()
print(f'{analysis_5.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

6

In [ ]:
f = 19
freeze_time_during_fire = True

analysis_6 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_6.run_all()

analysis_6.find_best_fitting_distributions()
print(f'{analysis_6.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


f = 76

analysis_7 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_7.run_all()

analysis_7.find_best_fitting_distributions()
print(f'{analysis_7.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

f = 305

analysis_8 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_8.run_all()

analysis_8.find_best_fitting_distributions()
print(f'{analysis_8.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 5))
r_squared_threshold = 0.97

ax = axes[0]
ax_title = 'Without time freezing'
label = f'f=19'
analysis_3.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_4.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
analysis_5.log_log_plot(ax, fig, color='orange', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

print('\n')

ax = axes[1]
ax_title = 'With time freezing'
label = f'f=19'
analysis_6.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_7.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
analysis_8.log_log_plot(ax, fig, color='orange', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold, show=True)

## Comparing different forest sizes with scaled spark frequencies

8

In [ ]:
L = 128
f = 125

freeze_time_during_fire = False

analysis_9 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_9.run_all()

analysis_9.find_best_fitting_distributions()
print(f'{analysis_9.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


f = 500

analysis_10 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_10.run_all()

analysis_10.find_best_fitting_distributions()
print(f'{analysis_10.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

f = 2000

analysis_11 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_11.run_all()

analysis_11.find_best_fitting_distributions()
print(f'{analysis_11.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 5))
r_squared_threshold = 0.97

ax = axes[0]
ax_title = '50 x 50 sized forest'
label = f'f=19'
analysis_3.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_4.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
analysis_5.log_log_plot(ax, fig, color='orange', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

print('\n')

ax = axes[1]
ax_title = '128 x 128 sized forest'
label = f'f=125'
analysis_9.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=500'
analysis_10.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=2000'
analysis_11.log_log_plot(ax, fig, color='orange', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold, show=True)

## Comparing freezing time with linear time for different spark frequencies while fixing average number of sparks instead of timesteps

7

In [ ]:
# Scaled down from 500_000
Average_sparks = 5_000

L = 50
f = 19

timesteps = int(Average_sparks * f)
instances = 1
freeze_time_during_fire = False
remember_history = False

analysis_12 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_12.run_all()

analysis_12.find_best_fitting_distributions()
print(f'{analysis_12.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


timesteps = int(Average_sparks * f)

analysis_13 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_13.run_all()

analysis_13.find_best_fitting_distributions()
print(f'{analysis_13.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

timesteps = int(Average_sparks * f)

analysis_14 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_14.run_all()

analysis_14.find_best_fitting_distributions()
print(f'{analysis_14.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

40

In [ ]:
freeze_time_during_fire = True

f = 19
timesteps = int(Average_sparks * f)

analysis_15 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_15.run_all()

analysis_15.find_best_fitting_distributions()
print(f'{analysis_15.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


f = 76
timesteps = int(Average_sparks * f)

analysis_16 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_16.run_all()

analysis_16.find_best_fitting_distributions()
print(f'{analysis_16.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

f = 305
timesteps = int(Average_sparks * f)

analysis_17 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_17.run_all()

analysis_17.find_best_fitting_distributions()
print(f'{analysis_17.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 5))
r_squared_threshold = 0.97

ax = axes[0]
ax_title = 'Without time freezing'
label = f'f=19'
analysis_12.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_13.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
analysis_14.log_log_plot(ax, fig, color='orange', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

print('\n')

ax = axes[1]
ax_title = 'With time freezing'
label = f'f=19'
analysis_15.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_16.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
title = 'Frequency fire sizes over all instances by fixing average number of fires'
analysis_17.log_log_plot(ax, fig, color='orange', ax_title=ax_title, title=title, label=label, scatter=False, r_squared_threshold=r_squared_threshold, show=True)

3

In [ ]:
# Scaled down from 5000
Average_sparks = 500

L = 128

f = 125
timesteps = int(Average_sparks * f)

analysis_18 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_18.run_all()

analysis_18.find_best_fitting_distributions()
print(f'{analysis_18.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')


f = 500
timesteps = int(Average_sparks * f)

analysis_19 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_19.run_all()

analysis_19.find_best_fitting_distributions()
print(f'{analysis_19.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

f = 2000
timesteps = int(Average_sparks * f)

analysis_20 =  Analyse(L, f, freeze_time_during_fire, remember_history, timesteps, instances)
analysis_20.run_all()

analysis_20.find_best_fitting_distributions()
print(f'{analysis_17.best_fitting_distributions[0] * 100}% of the model instances are best fitted by a power law distribution')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 5))
r_squared_threshold = 0.97

ax = axes[0]

ax_title = '50 x 50 sized forest'
label = f'f=19'
analysis_15.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=76'
analysis_16.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=305'
title = 'Frequency fire sizes over all instances by fixing average number of fires'
analysis_17.log_log_plot(ax, fig, color='orange', ax_title=ax_title, title=title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

print('\n')
ax = axes[1]

ax_title = '128 x 128 sized forest'
label = f'f=125'
analysis_18.log_log_plot(ax, fig, color='black', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=500'
analysis_19.log_log_plot(ax, fig, color='green', ax_title=ax_title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)

label = f'f=2000'
title = 'Frequency fire sizes over all instances by fixing average number of fires'
analysis_20.log_log_plot(ax, fig, color='orange', ax_title=ax_title, title=title, label=label, scatter=False, r_squared_threshold=r_squared_threshold)
